# **Task 1 (Conversation History and Summarization)**

In [3]:
!pip install openai

import openai
import os

**Create API Key**

In [4]:
from openai import OpenAI
import getpass


groq_api_key = getpass.getpass("Enter your Groq API key: ")

client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

Enter your Groq API key: ··········


In [5]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, what is AI?"}
    ]
)
print(response.choices[0].message.content)

Hello. Artificial Intelligence (AI) is a branch of computer science that focuses on creating intelligent machines that can think, learn, and behave like humans. AI involves the development of algorithms, statistical models, and computer programs that enable machines to perform tasks that typically require human intelligence, such as:

1. Learning: AI systems can learn from data and adapt to new situations.
2. Problem-solving: AI can analyze problems and come up with solutions.
3. Reasoning: AI can make decisions based on logic and evidence.
4. Perception: AI can interpret and understand data from sensors, images, and text.
5. Linguistic understanding: AI can understand and generate human language.

There are different types of AI, including:

1. **Narrow or Weak AI**: Designed to perform a specific task, like facial recognition, language translation, or playing chess.
2. **General or Strong AI**: A hypothetical AI that can perform any intellectual task that a human can.
3. **Superintel

Build Conversation History Manager

In [6]:
conversation_history = []

def add_message(role, content):
    """Add a message to the conversation history"""
    conversation_history.append({"role": role, "content": content})

add_message("user", "Hello, how are you?")
add_message("assistant", "I am fine, how can I help you?")
print(conversation_history)

[{'role': 'user', 'content': 'Hello, how are you?'}, {'role': 'assistant', 'content': 'I am fine, how can I help you?'}]


Truncate Conversation

In [7]:
def truncate_by_messages(history, n):
    """Keep only the last n messages"""
    return history[-n:]

def truncate_by_length(history, max_chars):
    """Keep conversation within character limit"""
    text = ""
    truncated = []
    for msg in reversed(history):
        if len(text) + len(msg["content"]) <= max_chars:
            truncated.insert(0, msg)
            text += msg["content"]
        else:
            break
    return truncated

print(truncate_by_messages(conversation_history, 1))
print(truncate_by_length(conversation_history, 50))

[{'role': 'assistant', 'content': 'I am fine, how can I help you?'}]
[{'role': 'user', 'content': 'Hello, how are you?'}, {'role': 'assistant', 'content': 'I am fine, how can I help you?'}]


Summarization

In [8]:
def summarize_history(history):
    global client
    summary_prompt = "Summarize this conversation briefly:\n\n"
    for msg in history:
        summary_prompt += f"{msg['role'].capitalize()}: {msg['content']}\n"

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes conversations."},
            {"role": "user", "content": summary_prompt}
        ]
    )
    return response.choices[0].message.content

Periodic Summarization

In [9]:
def run_conversation(user_input, k=3):
    global client
    add_message("user", user_input)

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=conversation_history
    )
    assistant_reply = response.choices[0].message.content
    add_message("assistant", assistant_reply)

    turns = len([m for m in conversation_history if m["role"] == "user"])
    if turns % k == 0:
        summary = summarize_history(conversation_history)
        conversation_history.clear()
        conversation_history.append({"role": "system", "content": f"Summary: {summary}"})

    return assistant_reply

In [10]:
print(run_conversation("Hi, what is AI?"))
print(run_conversation("Tell me how Groq works."))
print(run_conversation("Can you summarize so far?"))
print(conversation_history)  # see stored summary

AI stands for Artificial Intelligence, which refers to the development of computer systems that can perform tasks that typically require human intelligence. These tasks can include:

1. Learning: AI systems can learn from data and improve their performance over time.
2. Problem-solving: AI systems can identify and solve problems in a logical and systematic way.
3. Reasoning: AI systems can interpret and draw conclusions from data and rules.
4. Perception: AI systems can recognize and classify objects, images, and sounds.
5. Language understanding and generation: AI systems can understand and generate human language.

There are several types of AI, including:

1. Narrow or Weak AI: Designed to perform a specific task, such as a virtual assistant or a chatbot.
2. General or Strong AI: A hypothetical AI system that can perform any task that a human can.
3. Superintelligence: An AI system that significantly surpasses human intelligence in all areas.

The goal of AI is to create systems tha

**DEMO**

In [11]:
print("=== Normal Conversation ===")
print(run_conversation("Hi, what is AI?"))
print(run_conversation("Can you tell me how Groq works?"))
print(run_conversation("And why is summarization useful?"))

print("\n=== Full Conversation History ===")
for msg in conversation_history:
    print(f"{msg['role'].capitalize()}: {msg['content']}")

print("\n=== Truncate by Last 2 Messages ===")
truncated_n = truncate_by_messages(conversation_history, 2)
for msg in truncated_n:
    print(f"{msg['role'].capitalize()}: {msg['content']}")

print("\n=== Truncate by Max 150 Characters ===")
truncated_chars = truncate_by_length(conversation_history, 150)
for msg in truncated_chars:
    print(f"{msg['role'].capitalize()}: {msg['content']}")

print("\n=== Periodic Summarization (k=3) ===")

conversation_history.clear()

print(run_conversation("Hi there!", k=3))
print(run_conversation("I want to learn about AI.", k=3))
print(run_conversation("Can you summarize our chat?", k=3))

print("\n=== History After Summarization ===")
for msg in conversation_history:
    print(f"{msg['role'].capitalize()}: {msg['content']}")

=== Normal Conversation ===
AI (Artificial Intelligence) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

1. Learning: AI systems can learn from data, experiences, and feedback.
2. Problem-solving: AI systems can analyze problems, identify solutions, and make decisions.
3. Reasoning: AI systems can draw conclusions, make predictions, and understand cause-and-effect relationships.
4. Perception: AI systems can interpret and understand data from sensors, images, and speech.
5. Language understanding: AI systems can comprehend and generate human language.

AI has various types, including:

1. **Narrow or Weak AI**: Designed for a specific task, like a virtual assistant, image recognition, or language translation.
2. **General AI**: Intended to perform any intellectual task that a human can, but is currently not available.
3. **Superintelligence**: Far superior to human intelligence, with capabilities that could surpa

#  **Task 2 (JSON Schema Classification & Information Extraction)**

In [17]:
# JSON schema for extracting user details
schema = {
    "name": "extract_user_info",
    "description": "Extract user information from a chat",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "The user's full name"},
            "email": {"type": "string", "description": "The user's email address"},
            "phone": {"type": "string", "description": "The user's phone number"},
            "location": {"type": "string", "description": "The city or place where the user lives"},
            "age": {"type": "integer", "description": "The user's age"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

In [18]:
import json

def extract_user_info(user_message):
    """Extracts user details using Groq function calling"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",  # Groq model
        messages=[
            {"role": "system", "content": "You are an assistant that extracts user details in JSON format."},
            {"role": "user", "content": user_message}
        ],
        functions=[schema],  # pass the schema
        function_call={"name": "extract_user_info"}  # force structured output
    )

    # Parse the JSON response
    output = response.choices[0].message
    return output

# Example test
test_message = "Hi, I am Alice, my email is alice@test.com, phone is 9876543210, I live in Boston, and I am 30 years old."
result = extract_user_info(test_message)
print(result)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=FunctionCall(arguments='{"age":30,"email":"alice@test.com","location":"Boston","name":"Alice","phone":"9876543210"}', name='extract_user_info'), tool_calls=None)


In [19]:
test_message = "Hi, I am Alice, my email is alice@test.com, phone is 9876543210, I live in Boston, and I am 30 years old."
result = extract_user_info(test_message)
print(result)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=FunctionCall(arguments='{"age":30,"email":"alice@test.com","location":"Boston","name":"Alice","phone":"9876543210"}', name='extract_user_info'), tool_calls=None)


Final Demo

In [20]:
sample_chats = [
    "Hi, I’m John Doe. My email is john.doe@example.com, I am 28 years old, live in New York, and my phone number is 1234567890.",
    "Hello, my name is Sarah Lee. You can reach me at sarah.lee@mail.com. I’m 35, from Los Angeles, and my number is 9876543210.",
    "Hey, this is Raj. My email is raj.kumar@domain.com, phone 555666777, I’m 40 years old, and I stay in Chicago."
]

for i, chat in enumerate(sample_chats, start=1):
    print(f"\n--- Chat {i} ---")
    print("User:", chat)
    extracted = extract_user_info(chat)
    print("Extracted JSON:", extracted)


--- Chat 1 ---
User: Hi, I’m John Doe. My email is john.doe@example.com, I am 28 years old, live in New York, and my phone number is 1234567890.
Extracted JSON: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=FunctionCall(arguments='{"age":28,"email":"john.doe@example.com","location":"New York","name":"John Doe","phone":"1234567890"}', name='extract_user_info'), tool_calls=None)

--- Chat 2 ---
User: Hello, my name is Sarah Lee. You can reach me at sarah.lee@mail.com. I’m 35, from Los Angeles, and my number is 9876543210.
Extracted JSON: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=FunctionCall(arguments='{"age":35,"email":"sarah.lee@mail.com","location":"Los Angeles","name":"Sarah Lee","phone":"9876543210"}', name='extract_user_info'), tool_calls=None)

--- Chat 3 ---
User: Hey, this is Raj. My email is raj.kumar@domain.com, phone 555666777, I’m 40 years 